In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import random
import numpy as np
import spacy
import datasets
import torchtext
import tqdm
import evaluate

In [2]:
seed = 1234

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True

# 数据集 Dataset

## 1、添加一个Markdown单元格，在其中解释下方单元格的两行代码。
设置 os.environ['HF_ENDPOINT'] = \'https://hf-mirror.com' ，这样做具体改变了什么？
为什么要设置HF_ENDPOINT=\'https://hf-mirror.com'而非直接使用官方源？
dataset = datasets.load_dataset("bentrevett/multi30k") 这行代码具体完成了什么操作？

In [3]:
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'
dataset = datasets.load_dataset("bentrevett/multi30k")

Using the latest cached version of the dataset since bentrevett/multi30k couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at C:\Users\86134\.cache\huggingface\datasets\bentrevett___multi30k\default\0.0.0\4589883f3d09d4ef6361784e03f0ead219836469 (last modified on Tue Apr 29 12:17:27 2025).


### 对代码的解释

#### 1. 设置 `os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'`
- **具体改变**：  
  这行代码通过修改环境变量 `HF_ENDPOINT`，将 Hugging Face 的默认资源下载地址（`https://huggingface.co`）替换为镜像源 `https://hf-mirror.com`。所有后续通过 Hugging Face 库（如 `datasets`、`transformers`）发起的请求（如下载模型、数据集等）都会指向该镜像站点。

- **为何使用镜像而非官方源**：  
  1. **网络稳定性**：某些地区（如中国大陆）可能因网络限制无法稳定访问 Hugging Face 官方源，使用镜像可解决连接问题。  
  2. **下载速度**：镜像服务器通常部署在本地或优化网络路径，下载速度更快。  
  3. **合规性**：部分场景需通过合规镜像访问开源资源。

---

#### 2. `dataset = datasets.load_dataset("bentrevett/multi30k")`
- **具体操作**：  
  这行代码使用 Hugging Face 的 `datasets` 库从 Hub 加载名为 `multi30k` 的公开数据集（由用户 `bentrevett` 上传）。该数据集包含约 30,000 个英语-德语句对及对应图片描述，常用于多模态翻译任务。  
  - 执行时会自动完成以下操作：  
    1. 从镜像站下载数据集文件（若本地缓存不存在）。  
    2. 解析数据格式（如 CSV/JSON），返回结构化的 `Dataset` 或 `DatasetDict` 对象。  
    3. 支持直接用于训练或评估（如 `dataset["train"]` 访问训练集）。



## 2、运行下方的单元格。
你会看到数据集对象（一个DatasetDict）包含训练、验证和测试集，每个集合中的样本数量，以及每个集合中的特征（“en”和“de”）。


In [4]:
dataset


DatasetDict({
    train: Dataset({
        features: ['en', 'de'],
        num_rows: 29000
    })
    validation: Dataset({
        features: ['en', 'de'],
        num_rows: 1014
    })
    test: Dataset({
        features: ['en', 'de'],
        num_rows: 1000
    })
})

In [6]:
train_data, valid_data, test_data = (
    dataset["train"],
    dataset["validation"],
    dataset["test"],
)

## 3、运行下方的单元格。
我们可以索引每个数据集来查看单个示例。每个例子都有两个特征：“en”和“de”，是对应的英语和德语。


In [7]:
train_data[0]

{'en': 'Two young, White males are outside near many bushes.',
 'de': 'Zwei junge weiße Männer sind im Freien in der Nähe vieler Büsche.'}

接下来我们进行分词。英语/德语的分词较中文要直接，比如句子"good morning!会被分词为["good", "morning", "!"]序列。

下方的代码要成功安装en_core_web_sm和de_core_news_sm后才不会报错。

# 分词器 Tokenizers

In [9]:
en_nlp = spacy.load("en_core_web_sm")
de_nlp = spacy.load("de_core_news_sm")

## 4、运行下方的单元格。
我们可以使用.tokenizer方法调用每个spaCy模型的分词器，该方法接受字符串并返回Token对象序列。我们可以使用text属性从Token对象中获取字符串。


In [10]:
string = "What a lovely day it is today!"

[token.text for token in en_nlp.tokenizer(string)]

['What', 'a', 'lovely', 'day', 'it', 'is', 'today', '!']

## 5、添加一个Markdown单元格，在其中解释下方单元格的函数的作用。


In [11]:
def tokenize_example(example, en_nlp, de_nlp, max_length, lower, sos_token, eos_token):
    en_tokens = [token.text for token in en_nlp.tokenizer(example["en"])][:max_length]
    de_tokens = [token.text for token in de_nlp.tokenizer(example["de"])][:max_length]
    if lower:
        en_tokens = [token.lower() for token in en_tokens]
        de_tokens = [token.lower() for token in de_tokens]
    en_tokens = [sos_token] + en_tokens + [eos_token]
    de_tokens = [sos_token] + de_tokens + [eos_token]
    return {"en_tokens": en_tokens, "de_tokens": de_tokens}

## 函数 `tokenize_example` 的作用解析

### 函数功能概述
该函数用于对 **双语平行语料**（英语-德语）进行分词和预处理，生成适合序列到序列（Seq2Seq）模型（如机器翻译模型）训练的格式化 token 序列。其核心操作可分为以下步骤：

---

### 分步详解

#### 1. **输入参数说明**
| 参数         | 类型      | 作用                                                                 |
|--------------|-----------|--------------------------------------------------------------------|
| `example`    | dict      | 包含原始文本的字典，键为 `"en"`（英语）和 `"de"`（德语）               |
| `en_nlp`     | spaCy对象 | 英语分词器，用于切分英语句子（如 `spacy.load("en_core_web_sm")`）     |
| `de_nlp`     | spaCy对象 | 德语分词器，用于切分德语句子（如 `spacy.load("de_core_news_sm")`）    |
| `max_length` | int       | 限制分词后的最大长度（防止过长序列影响模型训练）                        |
| `lower`      | bool      | 是否将 token 转换为小写（统一文本大小写）                              |
| `sos_token`  | str       | 序列起始标记（如 `<sos>`），标识句子开始                                |
| `eos_token`  | str       | 序列结束标记（如 `<eos>`），标识句子结束                                |

#### 2. **处理流程**
1. **分词与截断**  
   - 使用 spaCy 分词器对英语和德语句子进行分词，生成单词列表  
   - 通过 `[:max_length]` 截断超过最大长度的部分  
   ```python
   en_tokens = [token.text for token in en_nlp.tokenizer(example["en"])][:max_length]
   de_tokens = [token.text for token in de_nlp.tokenizer(example["de"])][:max_length]

## 6、添加一个Markdown单元格，在其中解释下方单元格出现的\<sos>和\<eos>的含义，以及map函数的作用。


In [12]:
max_length = 1_000
lower = True
sos_token = "<sos>"
eos_token = "<eos>"

fn_kwargs = {
    "en_nlp": en_nlp,
    "de_nlp": de_nlp,
    "max_length": max_length,
    "lower": lower,
    "sos_token": sos_token,
    "eos_token": eos_token,
}

train_data = train_data.map(tokenize_example, fn_kwargs=fn_kwargs)
valid_data = valid_data.map(tokenize_example, fn_kwargs=fn_kwargs)
test_data = test_data.map(tokenize_example, fn_kwargs=fn_kwargs)


Map:   0%|          | 0/1014 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

## `<sos>`、`<eos>` 的含义与 `map` 函数的作用

---

### 一、特殊标记 `<sos>` 和 `<eos>` 的含义
| 标记         | 全称                     | 作用                                                                                   | 示例                          |
|--------------|--------------------------|--------------------------------------------------------------------------------------|-------------------------------|
| `<sos>`      | Start Of Sequence        | 标识序列的**起始位置**，告知模型从此处开始生成或解析内容                                 | `<sos> Two men are walking`   |
| `<eos>`      | End Of Sequence          | 标识序列的**结束位置**，帮助模型明确何时停止生成（避免无限生成）                          | `walking . <eos>`             |

**技术意义**：  
1. **序列边界控制**：为模型提供明确的输入/输出边界信号，是 Seq2Seq 模型的**标准实践**。  
2. **解码约束**：在推理阶段，模型通过检测 `<eos>` 终止生成（如翻译时输出完整句子后停止）。  
3. **注意力对齐**：帮助模型更好地学习序列间的对齐关系（如翻译中源语言与目标语言的词对应）。

---

### 二、`map` 函数的作用与实现逻辑

#### 1. **核心功能**  
将 `tokenize_example` 函数**批量应用**到数据集的所有样本上，实现**全量数据预处理**。

#### 2. **参数解析**  
```python
dataset.map(
    tokenize_example,         # 要应用的函数
    fn_kwargs = fn_kwargs     # 传递给函数的额外参数（字典形式）
)

## 7、运行下方的单元格
重新打印train_data\[0]，验证小写字符串列表以及序列标记的开始/结束符已被成功添加。


In [13]:
train_data[0]

{'en': 'Two young, White males are outside near many bushes.',
 'de': 'Zwei junge weiße Männer sind im Freien in der Nähe vieler Büsche.',
 'en_tokens': ['<sos>',
  'two',
  'young',
  ',',
  'white',
  'males',
  'are',
  'outside',
  'near',
  'many',
  'bushes',
  '.',
  '<eos>'],
 'de_tokens': ['<sos>',
  'zwei',
  'junge',
  'weiße',
  'männer',
  'sind',
  'im',
  'freien',
  'in',
  'der',
  'nähe',
  'vieler',
  'büsche',
  '.',
  '<eos>']}

# 词汇表 Vocabularies

下一个步骤是为源语言和目标语言构建词汇表，将词语映射为数字索引。比如"hello" = 1, "world" = 2, "bye" = 3, "hates" = 4。当向我们的模型提供文本数据时，我们使用词汇表作为look-up-table将字符串转换为标记，然后将标记转换为数字。“hello world”变成了“\[“hello”，“world”]”，然后变成了“\[1,2]”。

In [14]:
min_freq = 2
unk_token = "<unk>"
pad_token = "<pad>"

special_tokens = [
    unk_token,
    pad_token,
    sos_token,
    eos_token,
]

en_vocab = torchtext.vocab.build_vocab_from_iterator(
    train_data["en_tokens"],
    min_freq=min_freq,
    specials=special_tokens,
)

de_vocab = torchtext.vocab.build_vocab_from_iterator(
    train_data["de_tokens"],
    min_freq=min_freq,
    specials=special_tokens,
)

## 8、运行下方两个单元格
验证词汇表，分别打印英语词汇表和德语词汇表的前十个Token。
min_freq = 2
unk_token = "<unk>"
pad_token = "<pad>"

special_tokens = [
    unk_token,
    pad_token,
    sos_token,
    eos_token,
]

en_vocab = torchtext.vocab.build_vocab_from_iterator(
    train_data["en_tokens"],
    min_freq=min_freq,
    specials=special_tokens,
)

de_vocab = torchtext.vocab.build_vocab_from_iterator(
    train_data["de_tokens"],
    min_freq=min_freq,
    specials=special_tokens,
)

In [15]:
en_vocab.get_itos()[:10]

['<unk>', '<pad>', '<sos>', '<eos>', 'a', '.', 'in', 'the', 'on', 'man']

In [16]:
de_vocab.get_itos()[:10]

['<unk>', '<pad>', '<sos>', '<eos>', '.', 'ein', 'einem', 'in', 'eine', ',']

## 9、运行下方的单元格
使用get_stoi（stoi = "string to int "）方法获取指定的Token的索引。

In [17]:
en_vocab["the"]

7

In [18]:
assert en_vocab[unk_token] == de_vocab[unk_token]
assert en_vocab[pad_token] == de_vocab[pad_token]

unk_index = en_vocab[unk_token]
pad_index = en_vocab[pad_token]

In [19]:
en_vocab.set_default_index(unk_index)
de_vocab.set_default_index(unk_index)

词汇表的另一个有用特性是lookup_indices方法。它接受一个Token列表并返回一个索引列表。

## 10、运行下方的单元格
观察从Token列表到索引列表的转换。

In [20]:
tokens = ["i", "love", "watching", "crime", "shows"]
en_vocab.lookup_indices(tokens)

[956, 2169, 173, 0, 821]

对应的，lookup_tokens方法使用词汇表将索引列表转换回Token列表。

## 11、运行下方的单元格
观察从索引列表到Token列表的转换。


In [21]:
en_vocab.lookup_tokens(en_vocab.lookup_indices(tokens))

['i', 'love', 'watching', '<unk>', 'shows']

## 12、添加一个Markdown单元格，在其中解释为什么原本的"crime"被转换成了\<unk>。

在词表构建过程中，"crime" 被转换为 <unk>（未知词标记）的根本原因是：该词在训练数据中的出现频率低于 min_freq 的设定值，导致未被收录到词表中

## 13、添加一个Markdown单元格，在其中解释下方两个单元格中代码的作用。


In [22]:
def numericalize_example(example, en_vocab, de_vocab):
    en_ids = en_vocab.lookup_indices(example["en_tokens"])
    de_ids = de_vocab.lookup_indices(example["de_tokens"])
    return {"en_ids": en_ids, "de_ids": de_ids}

In [23]:
fn_kwargs = {"en_vocab": en_vocab, "de_vocab": de_vocab}

train_data = train_data.map(numericalize_example, fn_kwargs=fn_kwargs)
valid_data = valid_data.map(numericalize_example, fn_kwargs=fn_kwargs)
test_data = test_data.map(numericalize_example, fn_kwargs=fn_kwargs)

Map:   0%|          | 0/1014 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

### 代码作用解释

以下两个单元格的代码共同完成了**将文本数据转换为模型可处理的数字索引**的关键预处理步骤：

---

#### 单元格 1：数值化函数定义
```python
def numericalize_example(example, en_vocab, de_vocab):
    en_ids = en_vocab.lookup_indices(example["en_tokens"])  # 将英文词符转为索引
    de_ids = de_vocab.lookup_indices(example["de_tokens"])  # 将德文词符转为索引
    return {"en_ids": en_ids, "de_ids": de_ids}  # 返回数字化的样本

fn_kwargs = {"en_vocab": en_vocab, "de_vocab": de_vocab}  # 封装词汇表参数

## 14、运行下方的单元格
重新打印train_data\[0]，验证"en_ids" and "de_ids"被成功添加。


In [24]:
train_data[0]

{'en': 'Two young, White males are outside near many bushes.',
 'de': 'Zwei junge weiße Männer sind im Freien in der Nähe vieler Büsche.',
 'en_tokens': ['<sos>',
  'two',
  'young',
  ',',
  'white',
  'males',
  'are',
  'outside',
  'near',
  'many',
  'bushes',
  '.',
  '<eos>'],
 'de_tokens': ['<sos>',
  'zwei',
  'junge',
  'weiße',
  'männer',
  'sind',
  'im',
  'freien',
  'in',
  'der',
  'nähe',
  'vieler',
  'büsche',
  '.',
  '<eos>'],
 'en_ids': [2, 16, 24, 15, 25, 778, 17, 57, 80, 202, 1312, 5, 3],
 'de_ids': [2, 18, 26, 253, 30, 84, 20, 88, 7, 15, 110, 7647, 3171, 4, 3]}

Dataset类为我们处理的另一件事是将features转换为正确的类型。每个例子中的索引目前都是基本的Python整数。然而，为了在PyTorch中使用它们，它们需要转换为PyTorch张量。with_format方法将columns参数转换为给定的类型。这里，我们指定类型为“torch”，columns为“en_ids”和“de_ids”（我们想要转换为PyTorch张量的features）。默认情况下，with_format将删除任何不在传递给列的features列表中的features。我们希望保留这些features，这可以通过output_all_columns=True来实现。

In [25]:
data_type = "torch"
format_columns = ["en_ids", "de_ids"]

train_data = train_data.with_format(
    type=data_type, columns=format_columns, output_all_columns=True
)

valid_data = valid_data.with_format(
    type=data_type,
    columns=format_columns,
    output_all_columns=True,
)

test_data = test_data.with_format(
    type=data_type,
    columns=format_columns,
    output_all_columns=True,
)

## 15、运行下方的单元格
重新打印train_data[0]，验证“en_ids”和“de_ids”特征被转换为了张量。

In [26]:
train_data[0]

{'en_ids': tensor([   2,   16,   24,   15,   25,  778,   17,   57,   80,  202, 1312,    5,
            3]),
 'de_ids': tensor([   2,   18,   26,  253,   30,   84,   20,   88,    7,   15,  110, 7647,
         3171,    4,    3]),
 'en': 'Two young, White males are outside near many bushes.',
 'de': 'Zwei junge weiße Männer sind im Freien in der Nähe vieler Büsche.',
 'en_tokens': ['<sos>',
  'two',
  'young',
  ',',
  'white',
  'males',
  'are',
  'outside',
  'near',
  'many',
  'bushes',
  '.',
  '<eos>'],
 'de_tokens': ['<sos>',
  'zwei',
  'junge',
  'weiße',
  'männer',
  'sind',
  'im',
  'freien',
  'in',
  'der',
  'nähe',
  'vieler',
  'büsche',
  '.',
  '<eos>']}

# Data Loaders

数据准备的最后一步是创建Data Loaders。可以对它们进行迭代以返回一批数据，每一批数据都是一个字典，其中包含数字化的英语和德语句子作为PyTorch张量。

## 16、添加一个Markdown单元格，在其中解释下方两个单元格中的函数的作用。

In [27]:
def get_collate_fn(pad_index):
    def collate_fn(batch):
        batch_en_ids = [example["en_ids"] for example in batch]
        batch_de_ids = [example["de_ids"] for example in batch]
        batch_en_ids = nn.utils.rnn.pad_sequence(batch_en_ids, padding_value=pad_index)
        batch_de_ids = nn.utils.rnn.pad_sequence(batch_de_ids, padding_value=pad_index)
        batch = {
            "en_ids": batch_en_ids,
            "de_ids": batch_de_ids,
        }
        return batch

    return collate_fn

In [28]:
def get_data_loader(dataset, batch_size, pad_index, shuffle=False):
    collate_fn = get_collate_fn(pad_index)
    data_loader = torch.utils.data.DataLoader(
        dataset=dataset,
        batch_size=batch_size,
        collate_fn=collate_fn,
        shuffle=shuffle,
    )
    return data_loader

In [29]:
batch_size = 128

train_data_loader = get_data_loader(train_data, batch_size, pad_index, shuffle=True)
valid_data_loader = get_data_loader(valid_data, batch_size, pad_index)
test_data_loader = get_data_loader(test_data, batch_size, pad_index)

## 函数功能解析

### 整体目标
下方两个单元格的代码共同实现了**将预处理后的数据集转换为适合神经网络训练的批量张量数据**。核心解决两个问题：
1. **变长序列对齐**：通过填充使同批次内的序列等长
2. **数据加载优化**：高效生成可迭代的训练批次

---

### 函数详解

#### 函数1：`get_collate_fn(pad_index)`
```python
def get_collate_fn(pad_index):
    def collate_fn(batch):
        # 提取批次的英/德语句索引
        batch_en_ids = [example["en_ids"] for example in batch]
        batch_de_ids = [example["de_ids"] for example in batch]
        
        # 填充序列到等长（用pad_index填充）
        batch_en_ids = nn.utils.rnn.pad_sequence(batch_en_ids, padding_value=pad_index)
        batch_de_ids = nn.utils.rnn.pad_sequence(batch_de_ids, padding_value=pad_index)
        
        # 返回张量格式的批次数据
        return {"en_ids": batch_en_ids, "de_ids": batch_de_ids}
    
    return collate_fn


#### 函数2：`get_data_loader(dataset, batch_size, pad_index, shuffle=False)`
python
def get_data_loader(dataset, batch_size, pad_index, shuffle=False):
    collate_fn = get_collate_fn(pad_index)  # 获取填充函数
    return torch.utils.data.DataLoader(
        dataset=dataset,
        batch_size=batch_size,
        collate_fn=collate_fn,  # 应用填充逻辑
        shuffle=shuffle         # 是否打乱顺序
    )

# 构建模型

我们将分三部分构建模型。编码器，解码器和封装编码器和解码器的seq2seq模型。

# 编码器 Encoder

首先是编码器，它是一个2层的LSTM。

## 17、添加一个Markdown单元格，解释下方单元格中Encoder类的代码。
包括输入参数，核心组件（词嵌入层、LSTM层、Dropout层），forwad函数的处理流程，和输出。

In [30]:
class Encoder(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, n_layers, dropout):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=dropout)
        self.dropout = nn.Dropout(dropout)

    def forward(self, src):
        # src = [src length, batch size]
        embedded = self.dropout(self.embedding(src))
        # embedded = [src length, batch size, embedding dim]
        outputs, (hidden, cell) = self.rnn(embedded)
        # outputs = [src length, batch size, hidden dim * n directions]
        # hidden = [n layers * n directions, batch size, hidden dim]
        # cell = [n layers * n directions, batch size, hidden dim]
        # outputs are always from the top hidden layer
        return hidden, cell

## Encoder 类代码解析

### 类功能概述
该 `Encoder` 类实现了 **序列到序列（Seq2Seq）模型中的编码器部分**，负责将输入序列转换为上下文向量（Context Vector），核心功能包括：
1. **词嵌入表示**：将离散的词索引映射为连续向量
2. **上下文特征提取**：通过 LSTM 捕获序列的时序特征
3. **状态压缩**：输出最终隐藏状态作为序列的语义表示

---

### 输入参数说明
| 参数             | 类型   | 说明                                                                 |
|------------------|--------|--------------------------------------------------------------------|
| `input_dim`      | int    | 输入词表大小（即不同词符的数量）                                       |
| `embedding_dim`  | int    | 词嵌入向量的维度（通常 100-300）                                      |
| `hidden_dim`     | int    | LSTM 隐藏层的维度（决定特征提取能力，通常 256-1024）                   |
| `n_layers`       | int    | 堆叠的 LSTM 层数（深层网络可捕获复杂特征，通常 1-4）                    |
| `dropout`        | float  | Dropout 概率（防止过拟合，范围 0-1，仅在 `n_layers > 1` 时生效）        |

---

### 核心组件解析

#### 1. 词嵌入层 (`nn.Embedding`)
- **功能**：将离散的词索引转换为密集向量表示
- **数学形式**：`embedding_matrix = W ∈ ℝ^(input_dim × embedding_dim)`
- **示例**：若 `input_dim=10000`, `embedding_dim=300`，则生成 10,000 个 300 维向量

#### 2. LSTM 层 (`nn.LSTM`)
- **结构特性**：
  - 单层/多层堆叠结构
  - 每个时间步传递隐藏状态和细胞状态
- **参数维度**：
  - 输入维度：`embedding_dim`
  - 隐藏状态维度：`hidden_dim`
  - 层间 Dropout：当 `n_layers > 1` 时生效

#### 3. Dropout 层 (`nn.Dropout`)
- **作用位置**：在词嵌入后应用
- **功能**：随机屏蔽部分神经元，增强模型泛化能力
- **注意**：仅在模型训练时激活，评估模式自动关闭

---

### Forward 处理流程

#### 输入数据格式
- **src**: 形状为 `[src_length, batch_size]` 的张量
  - 示例：一个批次包含 32 个句子，每个句子长 20 词 → `shape=(20, 32)`

#### 处理步骤
1. **词嵌入转换**  
   ```python
   embedded = self.dropout(self.embedding(src))  
   # 输出形状: [src_length, batch_size, embedding_dim]

# 解码器 Decoder

接下来是解码器，它需要与编码器对齐，同样是一个2层的LSTM。

## 18、添加一个Markdown单元格，描述Decoder的工作流程。

In [32]:
class Decoder(nn.Module):
    def __init__(self, output_dim, embedding_dim, hidden_dim, n_layers, dropout):
        super().__init__()
        self.output_dim = output_dim
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.embedding = nn.Embedding(output_dim, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=dropout)
        self.fc_out = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, input, hidden, cell):
        # input = [batch size]
        # hidden = [n layers * n directions, batch size, hidden dim]
        # cell = [n layers * n directions, batch size, hidden dim]
        # n directions in the decoder will both always be 1, therefore:
        # hidden = [n layers, batch size, hidden dim]
        # context = [n layers, batch size, hidden dim]
        input = input.unsqueeze(0)
        # input = [1, batch size]
        embedded = self.dropout(self.embedding(input))
        # embedded = [1, batch size, embedding dim]
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        # output = [seq length, batch size, hidden dim * n directions]
        # hidden = [n layers * n directions, batch size, hidden dim]
        # cell = [n layers * n directions, batch size, hidden dim]
        # seq length and n directions will always be 1 in this decoder, therefore:
        # output = [1, batch size, hidden dim]
        # hidden = [n layers, batch size, hidden dim]
        # cell = [n layers, batch size, hidden dim]
        prediction = self.fc_out(output.squeeze(0))
        # prediction = [batch size, output dim]
        return prediction, hidden, cell

## Decoder 类工作流程解析

### 功能定位
Decoder 是 Seq2Seq 模型中的**序列生成器**，负责将编码器的上下文向量逐步解码为目标序列。其核心任务包括：
1. **逐步预测**：以自回归方式生成目标序列的每个词符
2. **状态传递**：维护 LSTM 的隐藏状态以保持序列连贯性
3. **概率映射**：将隐藏状态转换为目标词表的概率分布

---

### 输入参数说明
| 参数             | 类型   | 作用描述                                                                 |
|------------------|--------|------------------------------------------------------------------------|
| `output_dim`     | int    | 目标语言词表大小（决定最终输出维度）                                        |
| `embedding_dim`  | int    | 目标词嵌入维度（需与编码器一致或独立设定）                                   |
| `hidden_dim`     | int    | LSTM 隐藏层维度（需与编码器相同以实现状态兼容）                              |
| `n_layers`       | int    | LSTM 堆叠层数（通常与编码器对称）                                           |
| `dropout`        | float  | Dropout 概率（仅在训练时生效）                                              |

---

### 核心组件架构

#### 1. 词嵌入层 (`nn.Embedding`)
- 将目标词索引映射为连续向量
- 参数维度：`(output_dim, embedding_dim)`

#### 2. LSTM 层 (`nn.LSTM`)
- 接收嵌入向量与编码器的最终状态
- 参数维度：`input_size=embedding_dim`, `hidden_size=hidden_dim`

#### 3. 全连接层 (`nn.Linear`)
- 将 LSTM 输出映射到目标词表空间
- 参数维度：`(hidden_dim, output_dim)`

#### 4. Dropout 层 (`nn.Dropout`)
- 在词嵌入后应用，增强泛化能力

---

### Forward 处理流程

#### 输入数据格式
| 参数      | 形状                            | 来源说明                           |
|-----------|---------------------------------|----------------------------------|
| `input`   | `[batch_size]`                 | 当前时间步的输入词索引（通常是上一个预测词或真实词） |
| `hidden`  | `[n_layers, batch_size, hidden_dim]` | 编码器的最终隐藏状态或上一个时间步的隐藏状态 |
| `cell`    | `[n_layers, batch_size, hidden_dim]` | 编码器的最终细胞状态或上一个时间步的细胞状态 |

#### 处理步骤
1. **维度扩展**  
   ```python
   input = input.unsqueeze(0)  # [1, batch_size]
添加序列长度维度（单个时间步输入）

词嵌入转换

python
embedded = self.dropout(self.embedding(input))  # [1, batch_size, embedding_dim]
LSTM 状态更新

python
output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
输出形状：

output: [1, batch_size, hidden_dim]

hidden/cell: [n_layers, batch_size, hidden_dim]

概率分布生成

python
prediction = self.fc_out(output.squeeze(0))  # [batch_size, output_dim]
通过 softmax 可得各词的预测概率（实际应用时需额外添加 softmax 层）

# Seq2Seq

## 19、添加一个Markdown单元格，解释下方单元格中Seq2Seq类的代码。
包括forward函数的流程，以及teacher forcing机制。

In [33]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        assert (
            encoder.hidden_dim == decoder.hidden_dim
        ), "Hidden dimensions of encoder and decoder must be equal!"
        assert (
            encoder.n_layers == decoder.n_layers
        ), "Encoder and decoder must have equal number of layers!"

    def forward(self, src, trg, teacher_forcing_ratio):
        # src = [src length, batch size]
        # trg = [trg length, batch size]
        # teacher_forcing_ratio is probability to use teacher forcing
        # e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        batch_size = trg.shape[1]
        trg_length = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        # tensor to store decoder outputs
        outputs = torch.zeros(trg_length, batch_size, trg_vocab_size).to(self.device)
        # last hidden state of the encoder is used as the initial hidden state of the decoder
        hidden, cell = self.encoder(src)
        # hidden = [n layers * n directions, batch size, hidden dim]
        # cell = [n layers * n directions, batch size, hidden dim]
        # first input to the decoder is the <sos> tokens
        input = trg[0, :]
        # input = [batch size]
        for t in range(1, trg_length):
            # insert input token embedding, previous hidden and previous cell states
            # receive output tensor (predictions) and new hidden and cell states
            output, hidden, cell = self.decoder(input, hidden, cell)
            # output = [batch size, output dim]
            # hidden = [n layers, batch size, hidden dim]
            # cell = [n layers, batch size, hidden dim]
            # place predictions in a tensor holding predictions for each token
            outputs[t] = output
            # decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            # get the highest predicted token from our predictions
            top1 = output.argmax(1)
            # if teacher forcing, use actual next token as next input
            # if not, use predicted token
            input = trg[t] if teacher_force else top1
            # input = [batch size]
        return outputs

## Seq2Seq 类代码解析

### 整体功能
`Seq2Seq` 类实现了经典的**编码器-解码器架构**，通过协调 Encoder 和 Decoder 完成端到端的序列转换任务。其核心功能包括：
1. **全流程控制**：串联编码和解码阶段
2. **动态教学引导**：通过 Teacher Forcing 机制平衡真实标签与预测结果
3. **状态管理**：维护 LSTM 状态在时间步间的传递

---

### Forward 函数处理流程

#### 输入参数说明
| 参数                   | 形状                      | 作用                                                                 |
|------------------------|--------------------------|--------------------------------------------------------------------|
| `src`                  | `[src_len, batch_size]`  | 源语言序列（如英语句子）                                               |
| `trg`                  | `[trg_len, batch_size]`  | 目标语言序列（如德语句子）                                              |
| `teacher_forcing_ratio`| float (0-1)              | 使用真实标签作为解码器输入的概率                                         |

#### 处理步骤详解

1. **初始化输出容器**
   ```python
   outputs = torch.zeros(trg_length, batch_size, trg_vocab_size).to(device)
创建形状为 [目标序列长度, 批次大小, 目标词表大小] 的张量

示例：德语翻译任务中，若目标序列长 15，批次 32，词表 10000 → shape=(15, 32, 10000)

编码源序列

python
hidden, cell = self.encoder(src)  # 获取上下文向量
输出隐藏状态作为解码器的初始状态

设置初始输入

python
input = trg[0, :]  # 取目标序列首字符 <sos>
所有样本的初始输入均为起始符

自回归解码循环

python
for t in range(1, trg_length):
    output, hidden, cell = self.decoder(input, hidden, cell)
    outputs[t] = output  # 存储当前时间步预测结果
    
    # Teacher Forcing 决策
    teacher_force = random.random() < teacher_forcing_ratio
    top1 = output.argmax(1)  # 取预测概率最高的词
    input = trg[t] if teacher_force else top1

# 模型训练

模型初始化

## 20、添加注释
分别将“# 编码器初始化”，“# 解码器初始化”，“# Seq2Seq模型整合”这三行注释加到下方单元格中正确的位置

In [39]:
input_dim = len(de_vocab)
output_dim = len(en_vocab)
encoder_embedding_dim = 256
decoder_embedding_dim = 256
hidden_dim = 512
n_layers = 2
encoder_dropout = 0.5
decoder_dropout = 0.5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 编码器初始化
encoder = Encoder(
    input_dim,
    encoder_embedding_dim,
    hidden_dim,
    n_layers,
    encoder_dropout,
)

# 解码器初始化
decoder = Decoder(
    output_dim,
    decoder_embedding_dim,
    hidden_dim,
    n_layers,
    decoder_dropout,
)

# Seq2Seq模型整合
model = Seq2Seq(encoder, decoder, device).to(device)

权重初始化

In [35]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)


model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(7853, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(5893, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (fc_out): Linear(in_features=512, out_features=5893, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [36]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


print(f"The model has {count_parameters(model):,} trainable parameters")

The model has 13,898,501 trainable parameters


优化器 optimizer

In [37]:
optimizer = optim.Adam(model.parameters())

损失函数 Loss Function

In [38]:
criterion = nn.CrossEntropyLoss(ignore_index=pad_index)

Training Loop:

## 21、给下方单元格中的代码逐行加注释

In [45]:
def train_fn(
    model, data_loader, optimizer, criterion, clip, teacher_forcing_ratio, device
):
    # 设置模型为训练模式（启用dropout等）
    model.train()
    # 初始化累计损失
    epoch_loss = 0
    
    # 遍历数据加载器中的每个批次
    for i, batch in enumerate(data_loader):
        # 获取德语->英语的源/目标序列并转移到指定设备
        src = batch["de_ids"].to(device)  # 源语言序列（德语）
        trg = batch["en_ids"].to(device)  # 目标语言序列（英语）
        # src形状 = [序列长度, 批次大小]
        # trg形状 = [序列长度, 批次大小]

        # 清空上一批次的梯度
        optimizer.zero_grad()
        
        # 前向传播：获取模型输出
        output = model(src, trg, teacher_forcing_ratio)
        # output形状 = [目标序列长度, 批次大小, 目标词表大小]
        
        # 调整输出形状以计算损失：
        # 1. 忽略第一个时间步（<sos>不参与预测）
        # 2. 展平为二维张量 [(trg_len-1)*batch_size, vocab_size]
        output_dim = output.shape[-1]
        output = output[1:].view(-1, output_dim)
        
        # 调整目标序列形状：
        # 1. 忽略第一个时间步（预测从第二个词开始）
        # 2. 展平为一维张量 [(trg_len-1)*batch_size]
        trg = trg[1:].view(-1)
        
        # 计算交叉熵损失（自动处理ignore_index）
        loss = criterion(output, trg)
        
        # 反向传播计算梯度
        loss.backward()
        
        # 梯度裁剪防止梯度爆炸
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        # 更新模型参数
        optimizer.step()
        
        # 累计批次损失（自动解包标量值）
        epoch_loss += loss.item()
    
    # 返回该epoch的平均损失
    return epoch_loss / len(data_loader)

In [40]:
# def train_fn(
#     model, data_loader, optimizer, criterion, clip, teacher_forcing_ratio, device
# ):
#     model.train()
#     epoch_loss = 0
#     for i, batch in enumerate(data_loader):
#         src = batch["de_ids"].to(device)
#         trg = batch["en_ids"].to(device)
#         # src = [src length, batch size]
#         # trg = [trg length, batch size]
#         optimizer.zero_grad()
#         output = model(src, trg, teacher_forcing_ratio)
#         # output = [trg length, batch size, trg vocab size]
#         output_dim = output.shape[-1]
#         output = output[1:].view(-1, output_dim)
#         # output = [(trg length - 1) * batch size, trg vocab size]
#         trg = trg[1:].view(-1)
#         # trg = [(trg length - 1) * batch size]
#         loss = criterion(output, trg)
#         loss.backward()
#         torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
#         optimizer.step()
#         epoch_loss += loss.item()
#     return epoch_loss / len(data_loader)

Evaluation Loop:

In [41]:
def evaluate_fn(model, data_loader, criterion, device):
    model.eval()
    epoch_loss = 0
    with torch.no_grad():
        for i, batch in enumerate(data_loader):
            src = batch["de_ids"].to(device)
            trg = batch["en_ids"].to(device)
            # src = [src length, batch size]
            # trg = [trg length, batch size]
            output = model(src, trg, 0)  # turn off teacher forcing
            # output = [trg length, batch size, trg vocab size]
            output_dim = output.shape[-1]
            output = output[1:].view(-1, output_dim)
            # output = [(trg length - 1) * batch size, trg vocab size]
            trg = trg[1:].view(-1)
            # trg = [(trg length - 1) * batch size]
            loss = criterion(output, trg)
            epoch_loss += loss.item()
    return epoch_loss / len(data_loader)

# 模型训练

In [43]:
n_epochs = 1 # 因模型训练对计算资源要求较高，此处只设立了一轮训练。
clip = 1.0
teacher_forcing_ratio = 0.5

best_valid_loss = float("inf")

for epoch in tqdm.tqdm(range(n_epochs)):
    train_loss = train_fn(
        model,
        train_data_loader,
        optimizer,
        criterion,
        clip,
        teacher_forcing_ratio,
        device,
    )
    valid_loss = evaluate_fn(
        model,
        valid_data_loader,
        criterion,
        device,
    )
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), "tut1-model.pt")
    print(f"\tTrain Loss: {train_loss:7.3f} | Train PPL: {np.exp(train_loss):7.3f}")
    print(f"\tValid Loss: {valid_loss:7.3f} | Valid PPL: {np.exp(valid_loss):7.3f}")

100%|██████████| 1/1 [06:58<00:00, 418.81s/it]

	Train Loss:   8.671 | Train PPL: 5833.675
	Valid Loss:   8.669 | Valid PPL: 5821.597


# 模型验证

In [46]:
model.load_state_dict(torch.load("tut1-model.pt"))

<All keys matched successfully>

In [47]:
def translate_sentence(
    sentence,
    model,
    en_nlp,
    de_nlp,
    en_vocab,
    de_vocab,
    lower,
    sos_token,
    eos_token,
    device,
    max_output_length=25,
):
    model.eval()
    with torch.no_grad():
        if isinstance(sentence, str):
            tokens = [token.text for token in de_nlp.tokenizer(sentence)]
        else:
            tokens = [token for token in sentence]
        if lower:
            tokens = [token.lower() for token in tokens]
        tokens = [sos_token] + tokens + [eos_token]
        ids = de_vocab.lookup_indices(tokens)
        tensor = torch.LongTensor(ids).unsqueeze(-1).to(device)
        hidden, cell = model.encoder(tensor)
        inputs = en_vocab.lookup_indices([sos_token])
        for _ in range(max_output_length):
            inputs_tensor = torch.LongTensor([inputs[-1]]).to(device)
            output, hidden, cell = model.decoder(inputs_tensor, hidden, cell)
            predicted_token = output.argmax(-1).item()
            inputs.append(predicted_token)
            if predicted_token == en_vocab[eos_token]:
                break
        tokens = en_vocab.lookup_tokens(inputs)
    return tokens

In [48]:
sentence = test_data[0]["de"]
expected_translation = test_data[0]["en"]

sentence, expected_translation

('Ein Mann mit einem orangefarbenen Hut, der etwas anstarrt.',
 'A man in an orange hat starring at something.')

In [49]:
translation = translate_sentence(
    sentence,
    model,
    en_nlp,
    de_nlp,
    en_vocab,
    de_vocab,
    lower,
    sos_token,
    eos_token,
    device,
)

# 22、运行下方单元格，得到测试集第0个索引的翻译
因为epoch只进行了一轮，不会有好的效果的翻译。
感兴趣的同学可自行增加训练轮数，观察loss和翻译质量的变化。

In [50]:
translation

['<sos>',
 'multi',
 'flaps',
 'midair',
 'dustpan',
 'dustpan',
 'dustpan',
 'hairdo',
 'hair',
 'hair',
 'hairdo',
 'slender',
 'slender',
 'package',
 'helps',
 'helps',
 'finishing',
 'finishing',
 'newly',
 'newly',
 'rican',
 'rican',
 'rican',
 'perched',
 'observe',
 'observe']